In [1]:
import pickle
import os
from joblib import load
import pandas as pd
import scanpy as sc

/home/sfernandez/TFG/robustness_informed_TFG/.pixi/envs/ivaecuda/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/home/sfernandez/TFG/robustness_informed_TFG/.pixi/envs/ivaecuda/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/home/sfernandez/TFG/robustness_informed_TFG/.pixi/envs/ivaecuda/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_hdf from `anndata` is deprecated. Import anndata.io.read_hdf instead.
  warnings.warn(msg, FutureWarning)
/home/sfernandez/TFG/robustness_informed_TFG/.pixi/envs/ivaecuda/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_loom from `anndata` is deprecated. Import anndata.io.read_loom instead.
  warnings.warn(msg, F

In [3]:
# base_dir = os.path.abspath("..")  # Subir un nivel desde notebooks/
# path_encodings_layer_01 = os.path.join(base_dir, "path", "ivae_reactome", "encodings_layer-01_seed-00.pkl")
# path_encodings_layer_04 = os.path.join(base_dir, "path", "ivae_reactome", "encodings_layer-04_seed-00.pkl")
# path_metrics_seed = os.path.join(base_dir, "path", "ivae_reactome", "metrics-seed-00.pkl")
path_encodings_layer_01 = os.path.join("path", "ivae_reactome", "encodings_layer-01_seed-00.pkl")
path_encodings_layer_04 = os.path.join(b"path", "ivae_reactome", "encodings_layer-04_seed-00.pkl")
path_metrics_seed = os.path.join("path", "ivae_reactome", "metrics-seed-00.pkl")


x_train_encoded_01 = load(path_encodings_layer_01)
x_train_encoded_04 = load(path_encodings_layer_04)
metrics_seed = load(path_metrics_seed)
    
print(metrics_seed)  # Ver qué contiene

FileNotFoundError: [Errno 2] No such file or directory: '/home/sfernandez/TFG/robustness_informed_TFG/path/ivae_reactome/encodings_layer-01_seed-00.pkl'

In [4]:
x_train_encoded_01 = pd.DataFrame(x_train_encoded_01)
x_train_encoded_04 = pd.DataFrame(x_train_encoded_04)

NameError: name 'x_train_encoded_01' is not defined

In [21]:
# Crear objeto AnnData para análisis
adata = sc.AnnData(X=x_train_encoded_04)

/home/sfernandez/.local/lib/python3.10/site-packages/anndata/utils.py:307: UserWarning: X converted to numpy array with dtype object
  warnings.warn(f"{name} converted to numpy array with dtype {arr.dtype}")


In [23]:
layer_outputs = [layer.output for layer in x_train_encoded_04.layers]
activation_model = Model(inputs=x_train_encoded_04.input, outputs=layer_outputs)


AttributeError: 'DataFrame' object has no attribute 'layers'

In [16]:
adata = sc.AnnData(X=x_train_encoded_04.values)  # Convertir a numpy array

# Aplicar clustering y reducción de dimensionalidad
sc.pp.neighbors(adata, use_rep="X", random_state=0)
sc.tl.leiden(adata, random_state=0)
sc.tl.umap(adata, random_state=0)

In [17]:
import scanpy as sc
import pandas as pd

# Definir manualmente las condiciones si aún no lo has hecho
adata.obs["condition"] = ["control" if i < len(adata) // 2 else "stimulated" for i in range(len(adata))]

# Ranking de entidades o rutas diferenciales usando el test Wilcoxon
method = "wilcoxon"
sc.tl.rank_genes_groups(
    adata, groupby="condition", reference="control", key_added=method, method=method
)

# Extraer resultados de Wilcoxon
result = adata.uns[method]

# Confirmar que hay grupos válidos en el resultado
groups = result["names"].dtype.names
if not groups:
    print("No se encontraron resultados para los grupos especificados.")
else:
    # Crear DataFrame con los resultados deseados
    dacs = pd.DataFrame(
        {
            group + "_" + key: result[key][group]
            for group in groups
            for key in ["names", "scores", "pvals", "pvals_adj", "logfoldchanges"]
        }
    )

    # Mostrar las primeras 10 filas
    print(dacs.head(10))



  stimulated_names  stimulated_scores  stimulated_pvals  stimulated_pvals_adj  \
0              101           8.744868      2.232767e-18          6.013585e-16   
1              382           7.893230      2.944630e-15          2.379261e-13   
2               68           7.475380      7.698071e-14          4.784647e-12   
3              223           7.412989      1.234845e-13          6.651699e-12   
4              565           7.360659      1.830043e-13          9.241718e-12   
5              736           7.317310      2.529912e-13          1.202452e-11   
6              671           7.211325      5.541020e-13          2.238572e-11   
7              436           7.171064      7.441691e-13          2.863279e-11   
8              473           7.027605      2.101089e-12          7.716728e-11   
9              618           6.941428      3.881552e-12          1.363606e-10   

   stimulated_logfoldchanges  
0                  -0.134476  
1                  -0.532051  
2              

/home/sfernandez/.local/lib/python3.10/site-packages/scanpy/tools/_rank_genes_groups.py:455: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, "logfoldchanges"] = np.log2(
